In [3]:
#librerias
import pandas as pd
import numpy as np
import pyodbc

import matplotlib.pyplot as plot
import seaborn as sns



from datetime import datetime
from dateutil.relativedelta import relativedelta


import warnings
warnings.filterwarnings('ignore')


**1. Selección de datos**

In [4]:
#Conexión database
conn = pyodbc.connect(DRIVER='{ODBC Driver 17 for SQL Server}',
                      SERVER='dblatinkadwh.database.windows.net',
                      DATABASE='DWH_LaTinka',
                      UID='sandra.sanchez',
                      PWD='Latinka01*')
print(conn)


In [5]:
#No se tomó en cuenta los datos con target null
q0 = """SELECT *
        FROM [EDS].[TABLON_MODELO_FUGA_CLIENTE]
        WHERE FLG_FELICIDAD IS NOT NULL AND CODMES=202212"""
df = pd.read_sql_query(q0,conn)

#Cerrar conexión 
conn.close()

In [6]:
df.head(3)

,CODMES,ID_CLIENTE,VENTA_NETA_CASINO,VENTA_NETA_DEPORTE_VIRTUAL,VENTA_NETA_GANA_DIARIO,VENTA_NETA_GANAGOL,VENTA_NETA_KABALA,VENTA_NETA_KINELO,VENTA_NETA_RASPAYA,VENTA_NETA_TE_APUESTO,...,FECHA_ALTA_SISTEMA,FECHA_NACIMIENTO,PRODUCTO_PRINCIPAL,ESTADO_CLIENTE,FRECUENCY,MONETARY,RECENCY,FECHA_PRIMERA_ACCION,FECHA_ULTIMA_ACCION,FLG_FELICIDAD
0,202212,804013,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0,...,2019-04-27,1985-07-08,Te Apuesto,2. Recurrente,4. Habitual,5. Muy Bajo,1. Activo,2019-06-14,2023-04-15,No Feliz
1,202212,2113627,0.0,0.0,0.0,0.0,0.0,0.0,0.0,54.0,...,2022-12-03,2003-09-09,Te Apuesto,4. No Activo,5. Ocasional,5. Muy Bajo,3. Dormido 2,2022-12-03,2023-01-05,Feliz
2,202212,1741376,0.0,0.0,0.0,0.0,0.0,0.0,0.0,186.0,...,2022-06-05,1995-07-16,Te Apuesto,2. Recurrente,3. Frecuente,4. Bajo,1. Activo,2022-06-05,2023-04-19,Feliz


**2. Procesamiento**

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96223 entries, 0 to 96222
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   CODMES                      96223 non-null  int64  
 1   ID_CLIENTE                  96223 non-null  int64  
 2   VENTA_NETA_CASINO           96223 non-null  float64
 3   VENTA_NETA_DEPORTE_VIRTUAL  96223 non-null  float64
 4   VENTA_NETA_GANA_DIARIO      96223 non-null  float64
 5   VENTA_NETA_GANAGOL          96223 non-null  float64
 6   VENTA_NETA_KABALA           96223 non-null  float64
 7   VENTA_NETA_KINELO           96223 non-null  float64
 8   VENTA_NETA_RASPAYA          96223 non-null  float64
 9   VENTA_NETA_TE_APUESTO       96223 non-null  float64
 10  VENTA_NETA_TINKA            96223 non-null  float64
 11  NUM_VISITA                  96223 non-null  int64  
 12  TOTAL_RECARGA               96223 non-null  float64
 13  TOTAL_TRANSACCION_RECARGA   962

In [8]:
#Conversion de String a datetime
df['FECHA_ALTA_SISTEMA'] = pd.to_datetime(df['FECHA_ALTA_SISTEMA'], format = "%Y-%m-%d")
df['FECHA_NACIMIENTO'] = pd.to_datetime(df['FECHA_NACIMIENTO'], format = "%Y-%m-%d")
df['FECHA_PRIMERA_ACCION'] = pd.to_datetime(df['FECHA_PRIMERA_ACCION'], format="%Y-%m-%d")
df['FECHA_ULTIMA_ACCION'] = pd.to_datetime(df['FECHA_ULTIMA_ACCION'], format = "%Y-%m-%d")

In [9]:
df.isnull().sum()

CODMES                         0
ID_CLIENTE                     0
VENTA_NETA_CASINO              0
VENTA_NETA_DEPORTE_VIRTUAL     0
VENTA_NETA_GANA_DIARIO         0
VENTA_NETA_GANAGOL             0
VENTA_NETA_KABALA              0
VENTA_NETA_KINELO              0
VENTA_NETA_RASPAYA             0
VENTA_NETA_TE_APUESTO          0
VENTA_NETA_TINKA               0
NUM_VISITA                     0
TOTAL_RECARGA                  0
TOTAL_TRANSACCION_RECARGA      0
FECHA_ALTA_SISTEMA             0
FECHA_NACIMIENTO              24
PRODUCTO_PRINCIPAL             0
ESTADO_CLIENTE                 0
FRECUENCY                      0
MONETARY                       0
RECENCY                        0
FECHA_PRIMERA_ACCION           0
FECHA_ULTIMA_ACCION            0
FLG_FELICIDAD                  0
dtype: int64

In [10]:
#Eliminar filas con valores nulos 
df.dropna(inplace=True)

In [11]:
#Calular la Edad
df['EDAD'] = ((datetime.now() -df['FECHA_NACIMIENTO']).dt.days / 365).round(0).astype("int64")

In [12]:
df['DIAS_CON'] = (df['FECHA_ULTIMA_ACCION']-df['FECHA_PRIMERA_ACCION']).dt.days


In [13]:
#Reclass variables
df['PRODUCTO_PRINCIPAL'] = np.where(df['PRODUCTO_PRINCIPAL'] == 'Tinka', 1, 
                                            np.where(df['PRODUCTO_PRINCIPAL'] == 'Kabala', 2,
                                            np.where(df['PRODUCTO_PRINCIPAL'] == 'Te Apuesto', 3,
                                            np.where(df['PRODUCTO_PRINCIPAL'] == 'Casino', 4,
                                            np.where(df['PRODUCTO_PRINCIPAL'] == 'Gana Diario', 5,
                                            np.where(df['PRODUCTO_PRINCIPAL'] == 'Raspaya', 6,
                                            np.where(df['PRODUCTO_PRINCIPAL'] == 'Kinelo', 7,
                                            np.where(df['PRODUCTO_PRINCIPAL'] == 'Ganagol', 8,9))))))))

df['ESTADO_CLIENTE'] = np.where(df['ESTADO_CLIENTE'] == '4. No Activo',4,
                                        np.where(df['ESTADO_CLIENTE'] == '2. Recurrente',2,
                                        np.where(df['ESTADO_CLIENTE'] == '3. Esporadico',3,1)))

df['FRECUENCY'] = np.where(df['FRECUENCY'] == '1. Permanente',1,
                                   np.where(df['FRECUENCY'] == '1. Permanente',1,
                                   np.where(df['FRECUENCY'] == '2. Intensivo',2,
                                   np.where(df['FRECUENCY'] == '3. Frecuente',3,
                                   np.where(df['FRECUENCY'] == '4. Habitual',4,5)))))

df['MONETARY'] = np.where(df['MONETARY'] == '1. Muy Alto',1,
                          np.where(df['MONETARY'] == '2. Alto', 2,
                          np.where(df['MONETARY'] == '3. Medio', 3,
                          np.where(df['MONETARY'] == '4. Bajo', 4,5))))

df['RECENCY'] = np.where(df['RECENCY'] == '1. Activo', 1,
                          np.where(df['RECENCY'] == '2. Dormido 1', 2,
                          np.where(df['RECENCY'] == '3. Dormido 2',3,
                          np.where(df['RECENCY'] == '4. Perdido',4,5))))

df['FLG_FELICIDAD'] = np.where(df['FLG_FELICIDAD'] == 'No Feliz',0,1)

In [14]:
df.head(4)

,CODMES,ID_CLIENTE,VENTA_NETA_CASINO,VENTA_NETA_DEPORTE_VIRTUAL,VENTA_NETA_GANA_DIARIO,VENTA_NETA_GANAGOL,VENTA_NETA_KABALA,VENTA_NETA_KINELO,VENTA_NETA_RASPAYA,VENTA_NETA_TE_APUESTO,...,PRODUCTO_PRINCIPAL,ESTADO_CLIENTE,FRECUENCY,MONETARY,RECENCY,FECHA_PRIMERA_ACCION,FECHA_ULTIMA_ACCION,FLG_FELICIDAD,EDAD,DIAS_CON
0,202212,804013,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0,...,3,2,4,5,1,2019-06-14,2023-04-15,0,38,1401
1,202212,2113627,0.0,0.0,0.0,0.0,0.0,0.0,0.0,54.0,...,3,4,5,5,3,2022-12-03,2023-01-05,1,20,33
2,202212,1741376,0.0,0.0,0.0,0.0,0.0,0.0,0.0,186.0,...,3,2,3,4,1,2022-06-05,2023-04-19,1,28,318
3,202212,2124731,605.0,0.0,0.0,0.0,0.0,0.0,0.0,494.0,...,4,4,4,4,3,2022-12-06,2023-01-06,1,49,31


In [15]:
df = df.loc[:,['TOTAL_RECARGA', 'TOTAL_TRANSACCION_RECARGA','PRODUCTO_PRINCIPAL', 'FRECUENCY', 'MONETARY', 'RECENCY', 'EDAD', 'DIAS_CON', 'FLG_FELICIDAD']]


## División en objetivo y variables

In [16]:
X = df.drop(['FLG_FELICIDAD'], axis=1)
Y = df['FLG_FELICIDAD']

In [17]:
X.head(3)

,TOTAL_RECARGA,TOTAL_TRANSACCION_RECARGA,PRODUCTO_PRINCIPAL,FRECUENCY,MONETARY,RECENCY,EDAD,DIAS_CON
0,20.0,1,3,4,5,1,38,1401
1,35.0,2,3,5,5,3,20,33
2,50.0,1,3,3,4,1,28,318


In [18]:
Y.unique()

array([0, 1])

## Usando UBML

In [80]:
from sklearn.model_selection import train_test_split
X_train, X_test , Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=29)
print(len(X_train))
print(len(X_test))

76959
19240


In [20]:
from ubml.train_test import classification_train_test
metrics_performance, best_model = classification_train_test(x_train=X_train, x_test=X_test, y_train=Y_train, y_test=Y_test)
print(best_model)
metrics_performance


Random Forest Classifier


,accuracy score train,accuracy score test,f1 score train,f1 score test,recall score train,recall score test,jaccard score train,jaccard score test,precision score train,precision score test
Logistic Regression,0.681,0.680,0.807,0.806,0.994,0.994,0.676,0.675,0.679,0.677
KNN Classifier,0.754,0.642,0.829,0.752,0.893,0.811,0.708,0.602,0.774,0.701
SVM Classifier,0.670,0.668,0.803,0.801,1.000,1.000,0.670,0.668,0.670,0.668
Naive Bayes Classifier,0.445,0.445,0.344,0.340,0.217,0.214,0.207,0.205,0.827,0.829
Decision Tree Classifier,0.982,0.637,0.987,0.724,0.981,0.714,0.974,0.568,0.992,0.735
Random Forest Classifier,0.982,0.699,0.987,0.789,0.987,0.843,0.974,0.651,0.987,0.742


In [90]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit =(X_train, Y_train)


In [91]:
rf

RandomForestClassifier()

In [87]:
test_data = rf.predict(X_test)

NotFittedError: This RandomForestClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

Cargando y procesnado los datos de Test 2023

In [44]:
#Conexión database
conn1 = pyodbc.connect(DRIVER='{ODBC Driver 17 for SQL Server}',
                      SERVER='dblatinkadwh.database.windows.net',
                      DATABASE='DWH_LaTinka',
                      UID='sandra.sanchez',
                      PWD='Latinka01*')

In [45]:
# No se tomó en cuenta los datos con target null
q1 = """SELECT *
        FROM [EDS].[TABLON_MODELO_FUGA_CLIENTE]
        WHERE FLG_FELICIDAD IS NOT NULL AND CODMES=202301"""
df2 = pd.read_sql_query(q1, conn1)

#Cerrar conexión
conn1.close()


In [46]:
df2.head(4)

,CODMES,ID_CLIENTE,VENTA_NETA_CASINO,VENTA_NETA_DEPORTE_VIRTUAL,VENTA_NETA_GANA_DIARIO,VENTA_NETA_GANAGOL,VENTA_NETA_KABALA,VENTA_NETA_KINELO,VENTA_NETA_RASPAYA,VENTA_NETA_TE_APUESTO,...,FECHA_ALTA_SISTEMA,FECHA_NACIMIENTO,PRODUCTO_PRINCIPAL,ESTADO_CLIENTE,FRECUENCY,MONETARY,RECENCY,FECHA_PRIMERA_ACCION,FECHA_ULTIMA_ACCION,FLG_FELICIDAD
0,202301,773862,0.0,0.0,2.0,0.0,4.0,7.0,4.0,0.0,...,2019-03-20,1993-11-20,Kinelo,3. Esporadico,5. Ocasional,5. Muy Bajo,3. Dormido 2,2023-01-12,2023-01-14,No Feliz
1,202301,1168116,0.0,32.0,0.0,0.0,0.0,0.0,0.0,169.0,...,2020-10-08,1999-03-04,Te Apuesto,4. No Activo,3. Frecuente,5. Muy Bajo,2. Dormido 1,2022-12-10,2023-03-25,No Feliz
2,202301,2393968,0.0,301.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2023-01-13,1985-10-07,Deporte Virtual,3. Esporadico,5. Ocasional,4. Bajo,3. Dormido 2,2023-01-15,2023-01-15,No Feliz
3,202301,1130063,0.0,0.0,0.0,0.0,0.0,1.0,2.5,0.0,...,2020-09-06,1987-02-04,Tinka,3. Esporadico,5. Ocasional,5. Muy Bajo,2. Dormido 1,2023-01-20,2023-03-05,No Feliz


In [47]:
#Conversion de String a datetime
df2['FECHA_ALTA_SISTEMA'] = pd.to_datetime(df2['FECHA_ALTA_SISTEMA'], format = "%Y-%m-%d")
df2['FECHA_NACIMIENTO'] = pd.to_datetime(df2['FECHA_NACIMIENTO'], format = "%Y-%m-%d")
df2['FECHA_PRIMERA_ACCION'] = pd.to_datetime(df2['FECHA_PRIMERA_ACCION'], format="%Y-%m-%d")
df2['FECHA_ULTIMA_ACCION'] = pd.to_datetime(df2['FECHA_ULTIMA_ACCION'], format = "%Y-%m-%d")

In [48]:
# Eliminar filas con valores nulos
df2.dropna(inplace=True)


# Calular la Edad
df2['EDAD'] = ((datetime.now() - df2['FECHA_NACIMIENTO']
               ).dt.days / 365).round(0).astype("int64")

df2['DIAS_CON'] = (df2['FECHA_ULTIMA_ACCION']-df2['FECHA_PRIMERA_ACCION']).dt.days


In [49]:

# Reclass variables
df2['PRODUCTO_PRINCIPAL'] = np.where(df2['PRODUCTO_PRINCIPAL'] == 'Tinka', 1,
                                    np.where(df2['PRODUCTO_PRINCIPAL'] == 'Kabala', 2,
                                             np.where(df2['PRODUCTO_PRINCIPAL'] == 'Te Apuesto', 3,
                                                      np.where(df2['PRODUCTO_PRINCIPAL'] == 'Casino', 4,
                                                               np.where(df2['PRODUCTO_PRINCIPAL'] == 'Gana Diario', 5,
                                                                        np.where(df2['PRODUCTO_PRINCIPAL'] == 'Raspaya', 6,
                                                                                 np.where(df2['PRODUCTO_PRINCIPAL'] == 'Kinelo', 7,
                                                                                          np.where(df2['PRODUCTO_PRINCIPAL'] == 'Ganagol', 8, 9))))))))

df2['ESTADO_CLIENTE'] = np.where(df2['ESTADO_CLIENTE'] == '4. No Activo', 4,
                                np.where(df2['ESTADO_CLIENTE'] == '2. Recurrente', 2,
                                         np.where(df2['ESTADO_CLIENTE'] == '3. Esporadico', 3, 1)))

df2['FRECUENCY'] = np.where(df2['FRECUENCY'] == '1. Permanente', 1,
                           np.where(df2['FRECUENCY'] == '1. Permanente', 1,
                                    np.where(df2['FRECUENCY'] == '2. Intensivo', 2,
                                             np.where(df2['FRECUENCY'] == '3. Frecuente', 3,
                                                      np.where(df2['FRECUENCY'] == '4. Habitual', 4, 5)))))

df2['MONETARY'] = np.where(df2['MONETARY'] == '1. Muy Alto', 1,
                          np.where(df2['MONETARY'] == '2. Alto', 2,
                          np.where(df2['MONETARY'] == '3. Medio', 3,
                                   np.where(df2['MONETARY'] == '4. Bajo', 4, 5))))

df2['RECENCY'] = np.where(df2['RECENCY'] == '1. Activo', 1,
                         np.where(df2['RECENCY'] == '2. Dormido 1', 2,
                                  np.where(df2['RECENCY'] == '3. Dormido 2', 3,
                                           np.where(df2['RECENCY'] == '4. Perdido', 4, 5))))

df2['FLG_FELICIDAD'] = np.where(df2['FLG_FELICIDAD'] == 'No Feliz', 0, 1)


In [53]:
df2 = df2.loc[:,['TOTAL_RECARGA', 'TOTAL_TRANSACCION_RECARGA','PRODUCTO_PRINCIPAL', 'FRECUENCY', 'MONETARY', 'RECENCY', 'EDAD', 'DIAS_CON', 'FLG_FELICIDAD']]

In [58]:
X_2023 = df2.drop(columns=['FLG_FELICIDAD'], axis=1)

In [59]:
X_2023.head()

,TOTAL_RECARGA,TOTAL_TRANSACCION_RECARGA,PRODUCTO_PRINCIPAL,FRECUENCY,MONETARY,RECENCY,EDAD,DIAS_CON
0,20.0,1,7,5,5,3,29,2
1,160.0,3,3,3,5,2,24,105
2,300.0,1,9,5,4,3,38,0
3,40.0,1,1,5,5,2,36,44
4,25.0,1,6,4,5,2,29,483


In [65]:
Y_predict_2023 = model.predict(X_2023)

NotFittedError: This RandomForestClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.